In [ ]:
%pip install --upgrade transformers tensorflow


# off-the-shelf sentiment analysis pipeline

In [ ]:
# Load the off-the-shelf sentiment analysis pipeline

from transformers import pipeline

sentiment_analysis = pipeline("sentiment-analysis")

# Example texts 

texts = ["I loved this movie", "This movie was terrible", "This movie was the worst movie I have ever seen"]

# Perform sentiment analysis 

results_off_the_shelf = sentiment_analysis(texts)

# Print the results

for text, result in zip(texts, results_off_the_shelf):
    print(f"Text: {text}")
    print(f"Sentiment: {result['label']}")
    print(f"Confidence: {result['score']}")
    print("-------------------------")

# using the TextBlob library to perform sentiment analysis

In [16]:
%pip install textblob


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 626.3/626.3 kB 3.7 MB/s eta 0:00:00-:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 3.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


UsageError: Line magic function `%python` not found (But cell magic `%%python` exists, did you mean that instead?).


In [17]:
from textblob import TextBlob
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np


In [18]:

# Load the IMDb dataset
max_words = 10000  # Only consider the top 10,000 most frequent words
max_len = 200  # Pad sequences to a maximum length of 200 words

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_words)

# Convert IMDb integer sequences back to words for TextBlob
word_index = imdb.get_word_index()
reverse_word_index = {v: k for k, v in word_index.items()}

# Function to decode sequences back to text
def decode_review(text_sequence):
    return ' '.join([reverse_word_index.get(i - 3, '?') for i in text_sequence])

# Decode some test sequences to evaluate with TextBlob
X_test_texts = [decode_review(sequence) for sequence in X_test[:1000]]  # Evaluate 1000 reviews

# Prepare input data for deep learning models (LSTM/GRU)
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

# Use TextBlob for sentiment analysis on the test data
textblob_sentiments = []

for review in X_test_texts:
    blob = TextBlob(review)
    # TextBlob returns polarity: -1 (negative), 1 (positive), 0 (neutral)
    sentiment_score = blob.sentiment.polarity
    # Convert polarity to binary sentiment (similar to LSTM/GRU model)
    sentiment_label = 1 if sentiment_score > 0 else 0  # 1: positive, 0: negative
    textblob_sentiments.append(sentiment_label)

# TextBlob predicted labels
textblob_sentiments = np.array(textblob_sentiments)

# Evaluate TextBlob accuracy
textblob_accuracy = np.mean(textblob_sentiments == y_test[:1000])
print(f"TextBlob Sentiment Analysis Accuracy: {textblob_accuracy * 100:.2f}%")


TextBlob Sentiment Analysis Accuracy: 66.50%
